In [4]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openpyxl import load_workbook
import csv
import time
import os
import requests

In [10]:
# ==============================
# READ EXCEL
# ==============================
wb = load_workbook("data.xlsx")
sheet = wb.active

barcodes = []

for row in sheet.iter_rows(min_row=3, values_only=True):
    code = row[0]
    if code and str(code).strip():
        barcodes.append(str(code).strip())

print(f"Loaded {len(barcodes)} barcodes ✅\n")

Loaded 80 barcodes ✅



In [19]:
# ==============================
# DRIVER SETUP
# ==============================
options = uc.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = uc.Chrome(options=options)
wait = WebDriverWait(driver, 15)

cookie_handled = False
all_product_data = []

In [12]:
# ==============================
# FUNCTION: Handle cookies
# ==============================
def handle_cookies():
    global cookie_handled
    if not cookie_handled:
        try:
            cookie_btn = wait.until(
                EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, 'button[data-cookies="allow_all_cookies"]')
                )
            )
            driver.execute_script("arguments[0].click();", cookie_btn)
            print("✅ Cookies accepted\n")
            cookie_handled = True
            time.sleep(1)
        except:
            pass


# ==============================
# FUNCTION: Get first product link from listing page
# ==============================
def get_first_product_link(barcode):
    try:
        # Wait for either listing items to appear or "no results" message
        time.sleep(2)  # Give page time to load results
        
        # Check if there are any results
        listing_items = driver.find_elements(By.CSS_SELECTOR, ".listing-item__wrap")
        
        if not listing_items:
            print(f"   ⚠️  No products found for barcode: {barcode}\n")
            return None
        
        # Get the first listing item
        first_item = listing_items[0]
        
        # Get the product link from the title (try multiple selectors)
        try:
            title_link = first_item.find_element(By.CSS_SELECTOR, ".listing-item__name")
        except:
            # Try alternative selector
            title_link = first_item.find_element(By.CSS_SELECTOR, "a.listing-item__name, [data-link]")
        
        href = title_link.get_attribute("href")
        
        # If href is not in the link element, check data-link attribute
        if not href:
            href = title_link.get_attribute("data-link")
        
        # Remove fragment identifier (anything after #)
        if href and "#" in href:
            href = href.split("#")[0]
        
        print(f"   ✅ Found first product for barcode: {barcode}\n")
        return href
                
    except Exception as e:
        print(f"   ⚠️  Error getting product link: {str(e)[:100]}\n")
        return None


# ==============================
# FUNCTION: Scrape product details from product page with images
# ==============================
def scrape_product_details_enhanced(product_url, barcode):
    print(f"      Scraping: {product_url}")
    
    driver.get(product_url)
    time.sleep(3)
    
    # Create sanitized barcode for folder name
    sanitized_barcode = barcode.replace(" ", "_").replace("/", "-").replace("\\", "-")
    images_folder = f"images/{sanitized_barcode}"
    
    product_data = {
        "barcode": barcode,
        "product_url": product_url,
        "product_name": "",
        "price": "",
        "discount_percentage": "",
        "vat_percentage": "",
        "images_folder": images_folder,
        "images_downloaded": 0,
    }
    
    try:
        # Extract product name from h1
        try:
            h1_element = driver.find_element(By.CSS_SELECTOR, "h1.product-block__title")
            product_data["product_name"] = h1_element.text.strip()
        except:
            pass
        
        # Extract price
        try:
            price_element = driver.find_element(By.CSS_SELECTOR, ".product-block__price-new, .listing-item__price-new")
            product_data["price"] = price_element.text.strip()
        except:
            pass
        
        # Extract discount percentage
        try:
            discount_element = driver.find_element(By.CSS_SELECTOR, ".product-block__discount, .discount-percentage")
            product_data["discount_percentage"] = discount_element.text.strip()
        except:
            product_data["discount_percentage"] = "N/A"
        
        # Extract VAT percentage
        try:
            vat_element = driver.find_element(By.CSS_SELECTOR, ".product-block__inkl, .listing-item__inkl")
            vat_text = vat_element.text.strip()
            import re
            vat_match = re.search(r'(\d+)%', vat_text)
            if vat_match:
                product_data["vat_percentage"] = vat_match.group(1) + "%"
            else:
                product_data["vat_percentage"] = vat_text
        except:
            product_data["vat_percentage"] = "N/A"
        
        # Extract all description items dynamically
        try:
            description_items = driver.find_elements(By.CSS_SELECTOR, ".product-description__item")
            
            for item in description_items:
                try:
                    title_elem = item.find_element(By.CSS_SELECTOR, ".product-description__item-title")
                    value_elem = item.find_element(By.CSS_SELECTOR, ".product-description__item-value")
                    
                    title = title_elem.text.strip().replace(":", "").strip()
                    value = value_elem.text.strip()
                    
                    # Create a sanitized column name
                    column_name = title.replace(" ", "_").replace("[", "").replace("]", "").lower()
                    product_data[column_name] = value
                except:
                    continue
        except:
            pass
        
        # Download product images
        os.makedirs(images_folder, exist_ok=True)
        
        try:
            thumbnail_images = driver.find_elements(By.CSS_SELECTOR, ".product-gallery__image-list-item img")
            image_urls = []
            
            for img_elem in thumbnail_images:
                img_url = None
                
                # Try srcset first
                srcset = img_elem.get_attribute("srcset")
                if srcset:
                    parts = srcset.split(",")
                    for part in reversed(parts):
                        if "2x" in part or parts.index(part) == len(parts) - 1:
                            img_url = part.split()[0].strip()
                            break
                
                # Try data-srcset
                if not img_url:
                    data_srcset = img_elem.get_attribute("data-srcset")
                    if data_srcset:
                        parts = data_srcset.split(",")
                        for part in reversed(parts):
                            if "2x" in part or parts.index(part) == len(parts) - 1:
                                img_url = part.split()[0].strip()
                                break
                
                # Last resort: src
                if not img_url:
                    img_url = img_elem.get_attribute("src")
                
                if img_url and img_url.startswith("http"):
                    image_urls.append(img_url)
            
            # Download images
            downloaded_count = 0
            for idx, img_url in enumerate(image_urls, 1):
                try:
                    response = requests.get(img_url, timeout=10)
                    if response.status_code == 200:
                        ext = ".jpg"
                        if "." in img_url.split("/")[-1]:
                            url_filename = img_url.split("?")[0].split("/")[-1]
                            if "." in url_filename:
                                ext = "." + url_filename.split(".")[-1]
                        
                        filename = f"{images_folder}/image_{idx}{ext}"
                        with open(filename, "wb") as f:
                            f.write(response.content)
                        downloaded_count += 1
                except:
                    pass
            
            product_data["images_downloaded"] = downloaded_count
        except:
            pass
            
    except Exception as e:
        print(f"         ⚠️  Error scraping product: {e}")
    
    return product_data


# ==============================
# MAIN LOOP: Process ALL barcodes
# ==============================
from openpyxl import Workbook

print("\n🚀 Starting to process all barcodes...\n")

for idx, code in enumerate(barcodes, 1):
    print(f"[{idx}/{len(barcodes)}] Processing barcode: {code}")
    print("=" * 60)
    
    url = f"https://www.autodoc.parts/search?keyword={code}"
    driver.get(url)
    
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    handle_cookies()
    
    # Wait for page content to load
    time.sleep(3)
    
    # Get first product link only
    product_link = get_first_product_link(code)
    
    # Visit the product link and scrape details
    if product_link:
        product_data = scrape_product_details_enhanced(product_link, code)
        all_product_data.append(product_data)
        print(f"   ✅ Product scraped: {product_data.get('product_name', 'Unknown')[:50]}")
        print(f"   💾 Images: {product_data.get('images_downloaded', 0)} downloaded")
        print(f"\n✅ Completed barcode {code}\n")
    else:
        print(f"\n⚠️  No product found for barcode {code}\n")
    print("=" * 60)
    print()


# ==============================
# SAVE TO EXCEL
# ==============================
if all_product_data:
    output_file = "all_products_scraped.xlsx"
    
    # Collect all unique column names from all products
    all_columns = set()
    for product in all_product_data:
        all_columns.update(product.keys())
    
    # Sort columns to have consistent order (base columns first)
    base_columns = ["barcode", "product_url", "product_name", "price", "discount_percentage", 
                   "vat_percentage", "images_folder", "images_downloaded"]
    other_columns = sorted([col for col in all_columns if col not in base_columns])
    ordered_columns = base_columns + other_columns
    
    # Create Excel workbook
    wb_output = Workbook()
    ws = wb_output.active
    ws.title = "Products"
    
    # Write headers
    ws.append(ordered_columns)
    
    # Write data rows
    for product in all_product_data:
        row = [product.get(col, "") for col in ordered_columns]
        ws.append(row)
    
    # Auto-adjust column widths
    for column in ws.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = min(max_length + 2, 50)
        ws.column_dimensions[column_letter].width = adjusted_width
    
    wb_output.save(output_file)
    
    print(f"\n✅ SCRAPING COMPLETE!")
    print(f"   📊 Total products scraped: {len(all_product_data)}")
    print(f"   💾 Saved to: {output_file}")
    print(f"   📁 Total columns: {len(ordered_columns)}")
else:
    print("\n⚠️  No products were scraped")

driver.quit()
print("\n✅ All done! Browser closed.")


🚀 Starting to process all barcodes...

[1/80] Processing barcode: 34356790304 SK
✅ Cookies accepted

   ✅ Found first product for barcode: 34356790304 SK

      Scraping: https://www.autodoc.parts/ridex/8095160
   ✅ Product scraped: RIDEX 407W0062 Brake pad wear sensor
Rear Axle
   💾 Images: 4 downloaded

✅ Completed barcode 34356790304 SK


[2/80] Processing barcode: 34116769951


MaxRetryError: HTTPConnectionPool(host='localhost', port=60960): Max retries exceeded with url: /session/e2d8f8f3da831bbae7a2390ab610e460/url (Caused by NewConnectionError("HTTPConnection(host='localhost', port=60960): Failed to establish a new connection: [Errno 61] Connection refused"))

In [21]:
# ==============================
# TEST: Process FIRST barcode only - ENHANCED VERSION (Excel Output + Images)
# ==============================
from openpyxl import Workbook

# Reinitialize driver for testing - more robust cleanup
print("Initializing browser...")
try:
    driver.quit()
    del driver
    del wait
except:
    pass

options = uc.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = uc.Chrome(options=options)
wait = WebDriverWait(driver, 15)
cookie_handled = False

test_barcode = barcodes[0]
print(f"Testing with first barcode: {test_barcode}")
print("=" * 60)

try:
    url = f"https://www.autodoc.co.uk/spares-search?keyword={test_barcode}"

    driver.get(url)

    wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    handle_cookies()

    # Wait for page content to load
    time.sleep(3)

    # Get first product link
    product_link = get_first_product_link(test_barcode)

    # Visit the product link and scrape details
    if product_link:
        print(f"   Visiting product page...")
        driver.get(product_link)
        time.sleep(3)  # Give more time for product page to load
        
        # Create sanitized barcode for folder name
        sanitized_barcode = test_barcode.replace(" ", "_").replace("/", "-")
        images_folder = f"images/{sanitized_barcode}"
        
        product_data = {
            "barcode": test_barcode,
            "product_url": product_link,
            "product_name": "",
            "price": "",
            "discount_percentage": "",
            "vat_percentage": "",
            "images_folder": images_folder,
            "images_downloaded": 0,
        }
        
        try:
            # Extract product name from h1
            try:
                h1_element = driver.find_element(By.CSS_SELECTOR, "h1.product-block__title")
                product_data["product_name"] = h1_element.text.strip()
            except Exception as e:
                print(f"   ⚠️  Could not find product name: {e}")
            
            # Extract price
            try:
                price_element = driver.find_element(By.CSS_SELECTOR, ".product-block__price-new, .listing-item__price-new")
                product_data["price"] = price_element.text.strip()
            except Exception as e:
                print(f"   ⚠️  Could not find price: {e}")
            
            # Extract discount percentage
            try:
                discount_element = driver.find_element(By.CSS_SELECTOR, ".product-block__discount, .discount-percentage")
                product_data["discount_percentage"] = discount_element.text.strip()
            except:
                product_data["discount_percentage"] = "N/A"
            
            # Extract VAT percentage
            try:
                vat_element = driver.find_element(By.CSS_SELECTOR, ".product-block__inkl, .listing-item__inkl")
                vat_text = vat_element.text.strip()
                # Try to extract percentage from text like "price incl. 19% VAT"
                import re
                vat_match = re.search(r'(\d+)%', vat_text)
                if vat_match:
                    product_data["vat_percentage"] = vat_match.group(1) + "%"
                else:
                    product_data["vat_percentage"] = vat_text
            except:
                product_data["vat_percentage"] = "N/A"
            
            # Extract all description items dynamically
            try:
                description_items = driver.find_elements(By.CSS_SELECTOR, ".product-description__item")
                print(f"   Found {len(description_items)} description items")
                
                for item in description_items:
                    try:
                        title_elem = item.find_element(By.CSS_SELECTOR, ".product-description__item-title")
                        value_elem = item.find_element(By.CSS_SELECTOR, ".product-description__item-value")
                        
                        title = title_elem.text.strip().replace(":", "").strip()
                        value = value_elem.text.strip()
                        
                        # Create a sanitized column name
                        column_name = title.replace(" ", "_").replace("[", "").replace("]", "").lower()
                        product_data[column_name] = value
                    except Exception as e:
                        continue
            except Exception as e:
                print(f"   ⚠️  Error extracting description items: {e}")
            
            # Download product images
            print(f"\n   Downloading product images...")
            
            # Create folder for images
            os.makedirs(images_folder, exist_ok=True)
            
            image_urls = []
            try:
                # Get all thumbnail images from the gallery
                thumbnail_images = driver.find_elements(By.CSS_SELECTOR, ".product-gallery__image-list-item img")
                
                for img_elem in thumbnail_images:
                    # Try to get the highest resolution URL
                    img_url = None
                    
                    # First try srcset (highest resolution)
                    srcset = img_elem.get_attribute("srcset")
                    if srcset:
                        # srcset format: "url1 1x, url2 2x"
                        # Split by comma and get the 2x version (higher res)
                        parts = srcset.split(",")
                        for part in reversed(parts):  # Start from end to get 2x first
                            if "2x" in part or parts.index(part) == len(parts) - 1:
                                img_url = part.split()[0].strip()
                                break
                    
                    # If no srcset, try data-srcset
                    if not img_url:
                        data_srcset = img_elem.get_attribute("data-srcset")
                        if data_srcset:
                            parts = data_srcset.split(",")
                            for part in reversed(parts):
                                if "2x" in part or parts.index(part) == len(parts) - 1:
                                    img_url = part.split()[0].strip()
                                    break
                    
                    # Last resort: use src
                    if not img_url:
                        img_url = img_elem.get_attribute("src")
                    
                    if img_url and img_url.startswith("http"):
                        image_urls.append(img_url)
                
                print(f"   Found {len(image_urls)} images to download")
                
                # Download each image
                downloaded_images = []
                for idx, img_url in enumerate(image_urls, 1):
                    try:
                        response = requests.get(img_url, timeout=10)
                        if response.status_code == 200:
                            # Get file extension from URL or default to .jpg
                            ext = ".jpg"
                            if "." in img_url.split("/")[-1]:
                                url_filename = img_url.split("?")[0].split("/")[-1]
                                if "." in url_filename:
                                    ext = "." + url_filename.split(".")[-1]
                            
                            filename = f"{images_folder}/image_{idx}{ext}"
                            with open(filename, "wb") as f:
                                f.write(response.content)
                            downloaded_images.append(filename)
                            print(f"      ✅ Downloaded image {idx}/{len(image_urls)}: {filename} ({len(response.content)} bytes)")
                    except Exception as e:
                        print(f"      ⚠️  Failed to download image {idx}: {e}")
                
                # Update the count
                product_data["images_downloaded"] = len(downloaded_images)
                
            except Exception as e:
                print(f"   ⚠️  Error downloading images: {e}")
            
            print("\n✅ Product data extracted:")
            print(f"   Product Name: {product_data['product_name']}")
            print(f"   Price: {product_data['price']}")
            print(f"   Discount: {product_data['discount_percentage']}")
            print(f"   VAT: {product_data['vat_percentage']}")
            print(f"   Images Folder: {product_data['images_folder']}")
            print(f"   Images Downloaded: {product_data.get('images_downloaded', 0)}")
            print(f"   Total fields extracted: {len(product_data)}")
            
        except Exception as e:
            print(f"   ⚠️  Error scraping product: {e}")
        
        # Save to Excel
        test_output = "test_product_detailed.xlsx"
        wb_output = Workbook()
        ws = wb_output.active
        ws.title = "Product Data"
        
        # Write headers
        headers = list(product_data.keys())
        ws.append(headers)
        
        # Write data
        ws.append(list(product_data.values()))
        
        # Auto-adjust column widths
        for column in ws.columns:
            max_length = 0
            column_letter = column[0].column_letter
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = min(max_length + 2, 50)  # Cap at 50 characters
            ws.column_dimensions[column_letter].width = adjusted_width
        
        wb_output.save(test_output)
        
        print(f"\n✅ Saved detailed test product to {test_output}")
        print(f"   Columns saved: {list(product_data.keys())}")
    else:
        print(f"\n⚠️  No product found for barcode {test_barcode}")

    print("=" * 60)

except Exception as e:
    print(f"\n❌ Test failed with error: {e}")
    import traceback
    traceback.print_exc()
finally:
    # Close driver after test
    try:
        driver.quit()
        print("✅ Test complete. Browser closed.")
    except:
        print("⚠️  Browser was already closed.")

Initializing browser...
Testing with first barcode: 34356790304 SK
✅ Cookies accepted

   ✅ Found first product for barcode: 34356790304 SK

   Visiting product page...
   Found 9 description items

   Found 5 images to download
      ✅ Downloaded image 1/5: images/34356790304_SK/image_1.jpg (121914 bytes)
      ✅ Downloaded image 2/5: images/34356790304_SK/image_2.jpg (34922 bytes)
      ✅ Downloaded image 3/5: images/34356790304_SK/image_3.jpg (90063 bytes)
      ✅ Downloaded image 4/5: images/34356790304_SK/image_4.jpg (67880 bytes)
      ✅ Downloaded image 5/5: images/34356790304_SK/image_5.jpg (67700 bytes)

✅ Product data extracted:
   Product Name: RIDEX 407W0062 Brake pad wear sensor for BMW X3, X4
Rear Axle
   Price: £7. 59
   Discount: -42%
   VAT: 20%
   Images Folder: images/34356790304_SK
   Images Downloaded: 5
   Total fields extracted: 17

✅ Saved detailed test product to test_product_detailed.xlsx
   Columns saved: ['barcode', 'product_url', 'product_name', 'price', 'd